In [ ]:
import os
import pandas as pd
from underthesea import word_tokenize
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, f1_score

# **Data Loading**


In [52]:
def load_data(base_path='Train_Full'):
    data = []
    document_categories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    for category in tqdm(document_categories, desc="Loading categories"):
        category_path = os.path.join(base_path, category)
        print(category_path)
        for file_name in tqdm(os.listdir(category_path), desc=f"Loading {category}", leave=False):
            file_path = os.path.join(category_path, file_name)
            with open(file_path, 'r', encoding='utf-16') as f:
                text = f.read().strip()
            token_document = word_tokenize(text)
            process_document = ' '.join(token_document)
            data.append({'content': process_document, 'category': category})
    df = pd.DataFrame(data)
    return df, document_categories

In [53]:
train_df, categories = load_data('Train_Full')
test_df, _ = load_data('Test_Full')

Loading categories:   0%|          | 0/10 [00:00<?, ?it/s]

Train_Full\Chinh tri Xa hoi


Loading categories:  10%|█         | 1/10 [01:47<16:07, 107.50s/it]

Train_Full\Doi song


Loading categories:  20%|██        | 2/10 [03:17<12:57, 97.20s/it] 

...

Loading categories:  80%|████████  | 8/10 [27:04<07:57, 238.63s/it]

Test_Full\Van hoa


Loading categories:  90%|█████████ | 9/10 [31:57<04:15, 255.56s/it]

Test_Full\Vi tinh


Loading categories: 100%|██████████| 10/10 [34:41<00:00, 208.16s/it]


In [54]:
df.tail()

,content,category
5214,9.500 USD để du học thành du ngoạn Thủy đang h...,Chinh tri Xa hoi
5215,Nghiêm cấm giao đất trong hành lang đường Hồ C...,Chinh tri Xa hoi
5216,Chùm ảnh hoa Đà Lạt Thành phố Đà Lạt đã chọn 7...,Chinh tri Xa hoi
5217,Làm văn bằng qua ... tin nhắn Bạn muốn có chứn...,Chinh tri Xa hoi
5218,Du khách đến miền Trung xem lũ lụt Trong những...,Chinh tri Xa hoi


In [55]:
print(f"Categories: {categories}")

Categories: ['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh', 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa', 'Vi tinh']


In [56]:
print(f"Training Sample: {len(train_df)}")
print(f"Testing Sample: {len(test_df)}")

Training Sample: 33759
Testing Sample: 50373


In [57]:
category_to_id = {category: idx for idx, category in enumerate(categories)}
id_to_category = {idx: category for category, idx in category_to_id.items()}
print(id_to_category)

{0: 'Chinh tri Xa hoi', 1: 'Doi song', 2: 'Khoa hoc', 3: 'Kinh doanh', 4: 'Phap luat', 5: 'Suc khoe', 6: 'The gioi', 7: 'The thao', 8: 'Van hoa', 9: 'Vi tinh'}


In [58]:
train_df['category_id'] = train_df['category'].map(category_to_id)
test_df['category_id'] = test_df['category'].map(category_to_id)

In [59]:
train_df.head()

,content,category,category_id
0,Thành lập dự án POLICY phòng chống HIV / AIDS ...,Chinh tri Xa hoi,0
1,Hơn 16.000 khách đến vịnh Nha Trang Theo trực ...,Chinh tri Xa hoi,0
2,TPHCM : Khai trương dịch vụ lặn biển săn cá mậ...,Chinh tri Xa hoi,0
3,Du lịch VN sẽ có tư vấn nước ngoài Ông Phạm Từ...,Chinh tri Xa hoi,0
4,Quy chế tuyển sinh 2006 : Không làm tròn điểm ...,Chinh tri Xa hoi,0


In [60]:
# Combine for vocabulary building
all_texts = pd.concat([train_df['content'], test_df['content']])

In [61]:
train_texts = train_df['content'].values
train_categories = train_df['category_id'].values
test_texts = test_df['content'].values
test_categories= test_df['category_id'].values

# **📌 Document Classification Task**

Goal: Classify a document into one of the categories defined in the categories variable.

🔹 Dataset

- The dataset df consists of sequential text data.
- The order of words is crucial since it determines the overall meaning.

🔹 Model Choice

- ✅ LSTM (Long Short-Term Memory), a type of Recurrent Neural Network (RNN), is chosen to effectively capture the temporal dependencies in the text.


## Preprocessing

- Sequences matter: word order changes meaning ("good" ≠ "not good"), so LSTMs need sequential input.

- Tokenizer: maps words → integer IDs.

- Sequences: represent each document as ordered word IDs.

- Padding/Truncating: makes all sequences the same length for batching.

- Labels: converted to NumPy arrays for training.

- Converted labels into one-hot vectors for categorical crossentropy.


In [62]:
# Parameters
vocab_size = 20000  # VNTC has ~30k unique words typically
max_length = 300  # Max sequence length; articles are ~200-500 words --> use for pad/truncate when articles > or < max_length

In [63]:
# oov_token: handles words not in the vocabulary by replacing them with a special token (avoids errors)
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(all_texts)

In [64]:
# Convert to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [65]:
# Pad sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [66]:
# Convert labels to numpy arrays
train_categories = np.array(train_categories)
test_categories = np.array(test_categories)

## Build and Train the LSTM Model

**🧾 Model Summary**

### Model Architecture

1. **Embedding Layer**: turns word IDs into dense vectors (128 dimensions).
2. **BiLSTM (128 units, return sequences)**: captures forward + backward context.
3. **Dropout (0.3)**: reduces overfitting by randomly turn off 30% of neuron during traning time.
4. **BiLSTM (64 units)**: summarizes sequence.
5. **Dense (64, ReLU)**: learns higher-level features.
6. **Dense (softmax)**: outputs class probabilities, softmax is an activatin function for multiclass categories.

### Training

- **Optimizer**: Adam
- **Loss**: categorical crossentropy
- **Metric**: accuracy
- **Validation split**: 20% of training data used for validation.


In [67]:
num_classes = len(categories)

In [68]:
# One-hot encode labels for categorical crossentropy
train_categories_cat = to_categorical(train_categories, num_classes=num_classes)
test_categories_cat = to_categorical(test_categories, num_classes=num_classes)

In [69]:
# Build model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128))  # Embedding layer
model.add(Bidirectional(LSTM(128, return_sequences=True)))  # First LSTM
model.add(Dropout(0.3))  # Prevent overfitting
model.add(Bidirectional(LSTM(64)))  # Second LSTM
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
history = model.fit(
    train_padded, train_categories_cat,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
)

Epoch 1/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 589s 689ms/step - accuracy: 0.5251 - loss: 1.3431 - val_accuracy: 0.1706 - val_loss: 7.6583
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 418s 495ms/step - accuracy: 0.8368 - loss: 0.5263 - val_accuracy: 0.1719 - val_loss: 9.0609
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 492s 582ms/step - accuracy: 0.8686 - loss: 0.4324 - val_accuracy: 0.1739 - val_loss: 9.2876
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 463s 548ms/step - accuracy: 0.8935 - loss: 0.3462 - val_accuracy: 0.1749 - val_loss: 10.1652
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 449s 532ms/step - accuracy: 0.9137 - loss: 0.2905 - val_accuracy: 0.1739 - val_loss: 11.4785
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 468s 555ms/step - accuracy: 0.9140 - loss: 0.2763 - val_accuracy: 0.1696 - val_loss: 11.2222
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 447s 530ms/step - accuracy: 0.9326 - loss: 0.2221 - val_accuracy: 0.1731 - val_loss: 12.4813
Epoch 8/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 441s 522ms/step - accuracy: 0.93

## Evaluate the Model


In [71]:
# Predict on test set
test_preds = model.predict(test_padded)
test_preds_categories = np.argmax(test_preds, axis=1)


1575/1575 ━━━━━━━━━━━━━━━━━━━━ 150s 95ms/step


In [72]:
acc = accuracy_score(test_categories, test_preds_categories)
f1 = f1_score(test_categories, test_preds_categories, average='weighted')
print(f"Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")

Accuracy: 0.6816, F1-Score: 0.6151


# Testing on new data


In [73]:
def process_new_document(text):
    # Preprocess
    segmented_text = ' '.join(word_tokenize(text))
    sequence = tokenizer.texts_to_sequences([segmented_text])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')
    
    # Predict
    pred = model.predict(padded)
    pred_id = np.argmax(pred, axis=1)[0]
    return id_to_category[pred_id]

In [ ]:
article = "Bài báo về kinh tế Việt Nam năm 2025 với tăng trưởng GDP cao."
category = process_new_document(article)
print(f"Predicted category: {category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted category: Chinh tri Xa hoi


In [ ]:
article = "Tân binh Ngoại hạng Anh bị nghi đạo văn trong thư chia tay CLB."
category = process_new_document(article)
print(f"Predicted category: {category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted category: The thao


In [79]:
article = """Theo Interesting Engineering, công ty Điện khí Đông Phương của Trung Quốc lắp đặt turbine gió ngoài khơi 26 MW, vượt qua mẫu turbine 21,5 MW công ty Siemens Gamesa tại Đan Mạch để giành danh hiệu turbine mạnh nhất thế giới. Đông Phương thông báo nguyên mẫu turbine được đặt tại một cơ sở thử nghiệm và chứng nhận ngoài khơi tỉnh Phúc Kiến.

Mẫu turbine mới bao gồm hơn 30.000 bộ phận khác nhau. Cỗ máy lớn nhất toàn cầu cả về công suất và kích thước, có đường kính cánh rotor hơn 310 m và chiều cao trục là 185 m. Mỗi cánh rotor có chiều dài 153 m. Đây là thân trục turbine gió nặng nhất thế giới (500 tấn). Thân trục của turbine gió và 3 cánh rotor khổng lồ được chuyển đến cơ sở thử nghiệm vào đầu tháng 8/2025. Turbine gió được lắp đặt hoàn chỉnh sau 4 tuần.

Được thiết kế cho khu vực ngoài khơi có tốc độ gió từ 8 m/s trở lên, turbine có thể sản xuất 100 gigawatt-giờ điện hàng năm với sức gió trung bình 10 m/s. Sản lượng này đủ để cung cấp điện cho 55.000 hộ gia đình, giảm tiêu thụ 30.000 tấn than đá và giảm phát thải carbon dioxide 80.000 tấn. Hệ thống được thiết kế để chịu được gió mạnh tới 200 km/h.

Theo Heise, nguyên mẫu turbine gió 26 MW đang được thử nghiệm, bao gồm thử nghiệm nhằm phát hiện dấu hiệu mỏi. Trước đây, chỉ các bộ phận riêng lẻ như cánh rotor trải qua thử nghiệm tĩnh. Hiện nay, tất cả bộ phận phải hoạt động cùng lúc trong thực tế để chứng minh chúng có thể chịu được sử dụng liên tiếp.

Việc lắp đặt turbine 26 MW cho thấy Trung Quốc ngày càng thể hiện sức mạnh trên thị trường điện gió ngoài khơi. Năm nay, Trung Quốc dự kiến lắp đặt gần 75% turbine gió ngoài khơi mới trên thế giới. Lợi thế của nước này nằm ở chuỗi cung ứng tích hợp, hỗ trợ tài chính và chính sách của chính phủ, cải tiến công nghệ nhanh chóng. Ngược lại, các dự án tại Mỹ, châu Âu và Nhật Bản bị chậm dưới áp lực của chi phí cao, gián đoạn chuỗi cung ứng và trợ cấp giảm dần.

Các nhà sản xuất turbine lớn nhất của Trung Quốc bao gồm Đông Phương, Goldwind và Ming Yang Smart Energy, đang mở rộng ra thị trường nước ngoài. Một số tỉnh như Quảng Đông đặt mục tiêu tham vọng, hướng tới 17 GW gió ngoài khơi vào năm 2025, nhiều hơn bất kỳ quốc gia nào ngoài Trung Quốc. Đối với Đông Phương, turbine 26 MW là biểu tượng cho xu hướng phát triển những cỗ máy lớn và mạnh hơn, có thể thu được gió mạnh hơn ngoài khơi xa, giảm chi phí và thúc đẩy giới hạn công nghệ.
"""
category = process_new_document(article)
print(f"Predicted category: {category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted category: Khoa hoc
